The version below impliements the enumerate and does away with the append method. ALso implements .clear() for the buffer reset.

In [ ]:
import can # Required Module for CAN - PythonCAN
import time # used to pause the program to let the CAN adapter stablize
import numpy as np

from datetime import datetime

#Defining the filename at the start of the program. As the recording is likly to take 30 + mins enables me to set and forget. 
Filename = input("\nUnit Name -> ")

if Filename == "":
    Filename = "%s"%(datetime.today().strftime("%I.%M%p - %d %B %Y")) # Returns the current time and date in the format hh.mm AM/PM - day Month Year
else:
    Filename = "%s - %s"%(Filename, datetime.today().strftime("%I.%M %p - %d %B %Y"))


#Definding the Duration of the recording. 
def NumberCheck(Input, Command): # Checks that the value inputted it actually an integer. If not continues to ask until a Integer is inputted. 
        
    try:
        int(Input) # try to see if the value can be converted to a integer. If it can the function cycles to the next item in the list. 
        
    except ValueError: # if the above returns an error it asks again for the user to input a correct number. It then asks for a whole new list and re-initilizes the checking function. It then outputs the updated/corrected input list. 
        InputReattempt = input(Command)
        Input = NumberCheck(InputReattempt, Command) # performs the NumberCheck function again to confirm that a number was inputted the second time. 

    return Input

InputQuestion = "\nHow long do you want to record for in Mins ->  "
RecordingDuration = input(InputQuestion)

if RecordingDuration == "":
    RecordingDuration = "30"
else:
    RecordingDuration = NumberCheck(RecordingDuration,InputQuestion)

RecordingDuration = int(RecordingDuration)*60



#Defined CAN filters - CAN ID's I am only interested in. 
Filter = [{"can_id": 0x18FFFB14, "can_mask":0xFFFF00, "extended": True},{"can_id": 0x18FF4101, "can_mask":0xFFFF00, "extended": True},{"can_id": 0x18FF4001, "can_mask":0xFFFF00, "extended": True}]

# Bitrate for the MTU is 250000. The bitrate for a Cummins QSK is 500000

try:
    CANbus = can.interface.Bus(channel="PCAN_USBBUS1", interface="pcan", bitrate="250000", can_filters=Filter) # attempts to initialize the CAN interface. If it fails the except occurs below. 
    busstate = can.BusState.PASSIVE # defining the CAN bus. this should be set to PASSIVE if connecting to a real system. 
except can.CanError as error:
    print(f"Failed to Initialise the CAN bus due to {error}", end="\r", flush=True)
    exit(1) # exit of 1 means that the program ended with errors. 0 would mean it exited normally. 


time.sleep(2) # gives the bus time to intilise. 

CommandScale = 0.0025 # scale for the commandposition - in this case specific to the ProAct
FeedbackScale = 0.4
Offset = 0 # offset for the value. 

class CANMessageProcessor(can.Listener): # class that is called upon the interface reciving a CAN message. 
   
    global FeedbackScale
    global BufferFeedbackValueList
    global BufferPositionCommmand
    global CommandScale
    
    # Function below looks at the ID of the message recieved and depending on if its from the ProAct (Feedback) or the ECU (Command) processes it into different lists. 
    def on_message_received(self, Message):
        global FeedbackScale
        global BufferFeedbackValueList
        global BufferPositionCommmand
        global CommandScale
        global MessageIDList

        MessageID = str(Message.arbitration_id) # converting the CAN ID into a string so that I can compare it using indexing with the PGN of a ProAct Feedback Message. 
                
        if MessageID[0:6] == "419429": # comparing the message with the first part of the decimal version of the CAN ID (in string format due to indexing) (Basically doesnt look at the last byte which is the origin ID)

            TimeStamp = format(Message.timestamp,".3f")
            ID = Message.arbitration_id
            DesiredPosition = format(FeedbackScale*Message.data[1],".1f")
            ActualPosition = format(FeedbackScale*Message.data[0],".1f")
            BufferFeedbackValueList.append([ID,TimeStamp,DesiredPosition,ActualPosition]) # append to a list, more efficient to append to a list and then convert to a np.array at the end. 

        else:
            TimeStamp = format(Message.timestamp,".3f")
            ID = Message.arbitration_id
            CommandedPosition = CommandScale*int.from_bytes(Message.data[0:2], byteorder="little") # automatically converts the little Endian number to the correct int value. Commanded scale does the conversion. 
            BufferPositionCommmand.append([ID,TimeStamp,CommandedPosition])


        

BufferFeedbackValueList = []
BufferPositionCommmand = []

CANMessageProcessorInstance = CANMessageProcessor()

CANInterfaceRecievedMessageNotifier = can.Notifier(CANbus, [CANMessageProcessorInstance]) # this initiates the Notifier that sends recieved messages to the feedbacklistener class. 

#RecordingLength = 36000 # Length of the recording. The cycle time for a ProAct feedback message is 100ms. Thus 36000 is 30 mins worth. 



while len(BufferFeedbackValueList)<2:
    pass
else:
    
    EndTime = round(float(BufferFeedbackValueList[0][1])+int(RecordingDuration),3)
    StartTime = float(BufferFeedbackValueList[0][1])

while float(BufferFeedbackValueList[-1][1])<=EndTime: # Function continues until the Feedback message buffer list reaches the defined length above. Whilst doing it it updates showing a running percentage. 
    PercentageComplete = format(((float(BufferFeedbackValueList[-1][1])-StartTime)/RecordingDuration)*100,".1f")
    print(f"{PercentageComplete}% Complete", end="\r" , flush=True)    # "\r" ensures that the print function overwrites itself. It therefore looks like its updating the value rather than writing a new one. 

else: # When its collected suffient data, stop the CAN interface, convert the buffer lists to Numpy arrays and then save them. 

    CANInterfaceRecievedMessageNotifier.stop()

    CANbus.shutdown()

    FeedbackOutputArray = np.asarray(BufferFeedbackValueList)
    PositionDemandOutputArray = np.asarray(BufferPositionCommmand)

    np.save("ProActFeedbackPositionRecordings/ProAct Feedback Position Recording - %s"%(Filename), FeedbackOutputArray, allow_pickle=True)
    np.save("ProActCommandedPositionRecordings/ProAct Commanded Position Recording - %s"%(Filename), PositionDemandOutputArray, allow_pickle=True)